In [66]:
# Import libraries
import os
import json

In [67]:
project_path = '/Users/danyentezari/Documents/projects_moved/bignumber.xyz/public/bignumber-material'

In [68]:
# Example with loading a JSON file
def load(path):
    # Open the JSON file
    with open(path, 'r') as file:
        # Load the JSON data
        jsonObj = json.load(file)
        return jsonObj
 
def save(obj, path):   
    json_data = json.dumps(obj)

    with open(path, 'w') as file:
        # Write JSON data to the file
        json.dump(obj, file)
        print('Saved file')

In [69]:
# Process BigNumber Course Assessment (.ba)
def process_assessment(path):
    assessmentObj = dict({
        "fileName": "",
        "file": "",
        "title": "",
        "duration": "",
        "quiz": "",
        "url": "",
        "assessmentDetails": dict({
            "organization": []
        })  
    })
        
    l = dict(
        {
            "qt": "questionTitle",
            "t": "type",
            "q": "question",
            "o": "options",
            "a": "answerKey",
            "h": "hint",
            "c": "choice",
            "b": "body"
        }
    )

    template_preprocess = -1
    # Open the file in read mode
    with open(path, 'r') as file:
        # Read each line and print it if it's not empty
        for line in file:
            line = line.strip()
            if line:
                
                # Assessment setup
                if line == '--- template_end ---':
                    template_preprocess = 0
                    
                if template_preprocess == 1:
                    # print(line)
                    key, value = line.split("::")
                    # pair = [key.strip(), value.strip()]
                    key = key.strip()
                    value = value.strip()
                    
                    # Casting is done is done at playlist level
                    if(value == "true"): value = True
                    if(value == "false"): value = False
                    
                    if type(assessmentObj[key]) == str:
                        assessmentObj[key] = value
                        
                if line == '--- template_start ---':
                    template_preprocess = 1
            
                
                # Assessment organization
                # Start processing questions, answers, hints after template_end..
                if template_preprocess == 0:
                    if line != "--- template_end ---":
                        detail = line.split('::')
                            
                        if len(detail) > 1:
                                                    
                            k = detail[0].strip()
                            v = detail[1].strip()

                            # Casting is done is done at playlist level                          
                            if(v == "true"):  v = True
                            if(v == "false"):  v = False
                            
                            if l[k] == "questionTitle":
                                question_type = ""
                                new_question = dict({
                                    "questionTitle": "",
                                    "details": dict({
                                        "options": [],
                                        "answerKey": "",
                                        "hint": ""
                                    })
                                })
                                append_next = 1
                            
                            # l[] is for the lookup
                            if l[k] == "questionTitle" :
                                new_question[l[k]] = v
                            
                            elif l[k] == "type":
                                question_type = v
                                
                                # Setup options for the question
                                if len(new_question['details']['options']) == 0:
                                    new_question['details']['options'] = []
                                
                                # If simple text input, append options straightaway
                                if question_type == "text":
                                    new_question['details']['options'].append(
                                        dict({
                                            "type": question_type,
                                            "text": ""
                                        }))
                            
                            elif l[k] == "options":
                                if question_type == "radio":
                                    new_question['details']['options'].append(
                                        dict({
                                            "type": question_type,
                                            "choices": []
                                        }))
                                                                    
                                
                            elif l[k] == "choice":
                                new_question['details']['options'][0]['choices'].append(
                                    {"text": v}
                                )
                            
                            else: 
                                new_question['details'][l[k]] = v
                            
                            if append_next == 1:
                                append_next = 0
                                assessmentObj['assessmentDetails']['organization'].append(new_question)
                      
    return assessmentObj, assessmentObj['fileName']

process_assessment(f'./mod1/file-1-4_quiz_1-1.ba')

({'fileName': 'Assessment-1',
  'file': 'Assessment-1',
  'title': 'Assessment 1',
  'duration': '2m',
  'quiz': True,
  'url': 'spr/Assessment-1',
  'assessmentDetails': {'organization': [{'questionTitle': 'Q1',
     'details': {'options': [{'type': 'radio',
        'choices': [{'text': '4'}, {'text': '3'}, {'text': '0'}]}],
      'answerKey': '4',
      'hint': "Don't be silly",
      'question': 'What is the sum of 2+2?'}},
    {'questionTitle': 'Q2',
     'details': {'options': [{'type': 'text', 'text': ''}],
      'answerKey': 'Dany',
      'hint': "C'mon now!",
      'question': 'Enter your name'}},
    {'questionTitle': 'Q3',
     'details': {'options': [{'type': 'text', 'text': ''}],
      'answerKey': '4',
      'hint': 'Wrong!',
      'question': 'What is $f(x)^2$; when $x$ = 2?'}}]}},
 'Assessment-1')

In [70]:
import subprocess


# Process BigNumber Course Videoplaylist (.bpls)
def process_playlist_section(path):
    sectionObj = dict(
        {
            "fileName": "",
            "section": "",
            "children": []
        }
    )
    
    mod = path.split(".")[1][-1]
    
    course = "SPR"
    course_file_path = f'bignumber-material/{course}/mod{mod}'
    url_prefix = f'http://localhost:3001'
    url_prefix_math = f'http://localhost:3000'

    template_preprocess = -1
    i_file = 0
    
    subprocess.run(
        f"if [ -d './mod{mod}' ]; then echo 'The folder exists.'; else mkdir -p './mod{mod}'; fi",
        shell=True,
        check=True
    )
    
    
    with open(path, 'r') as file:
        for line in file:
            line = line.strip()
                
            # Section setup
            if line == '--- template_end ---':
                template_preprocess = 0
                
            if template_preprocess == 1:
                key, value = line.split("::")
                # pair = [key.strip(), value.strip()]
                key = key.strip()
                value = value.strip()
                
                if(value == "true"): value = True
                if(value == "false"): value = False
                
                if type(sectionObj[key]) == str:
                    sectionObj[key] = value    
                
            if line == '--- template_start ---':
                template_preprocess = 1
                
                
            # Playlist processing
            # Start processing file, url, title...
            if template_preprocess == 0:
                if line != "--- template_end ---":
                    # detail = line.split("::")
                    
            
                    if len(line) > 1:
                          
                        if line[0:1] != "_":
                            append_next = 1
                            child = dict({})
                        
                        detail = line.split("::")      
                        
                        if len(detail) > 1 and line[0:1] != "_":
                            k = detail[0].strip()
                            v = detail[1].strip()   
                            
                            i_file += 1
                            # new_filename = f'file-{mod}-{i_file}_{k.lower().replace(" ", "_")}'
                            title_filename = f'{k.lower().replace(" ", "_")}'
                            subprocess.run(["touch", f'./mod{mod}/{title_filename}.md'], check=True)
                            
                            
                            if v[0:8] == "math_viz":    
                               
                                math_module = v.split('\'')[1]
                                child['file'] = f'{math_module}'
                                child['title'] = k
                                child['filePath'] = f'{url_prefix_math}/{math_module}'
                                child['url'] = f'{url_prefix_math}/{math_module}'
                                child['math_viz'] = True
                                
                                child['tutorial'] = f'{url_prefix}/{course_file_path}/{title_filename}.md'
                            
                            elif v[0:5] == "vimeo":
                                vimeo_module = v.split('\'')[1]
                                child['file'] = f'{vimeo_module}'
                                child['title'] = k
                                child['filePath'] = f'{vimeo_module}'
                                child['url'] = f'{vimeo_module}'
                                child['vimeo'] = True
                                
                            elif v[0:2] == "ba": # assessment 
                                # assessment_module = v.split('\'')[1]
                                
                                child['file'] = f'{title_filename}.ba'
                                child['title'] = k
                                child['filePath'] = f'{course_file_path}/{title_filename}.ba'
                                child['url'] = f'{url_prefix}/{course_file_path}/{title_filename}.ba'
                                child['assessment'] = f"load('./{title_filename}.json')"
                                
                                subprocess.run(["touch", f'./mod{mod}/{title_filename}.ba'], check=True)
                            
                            elif v == "notebook":
                                child['file'] = f'{title_filename}.ipynb'
                                child['title'] = k
                                child['filePath'] = f'{course_file_path}/{title_filename}.ipynb'
                                child['url'] = f'{url_prefix}/{course_file_path}/{title_filename}.ipynb'
                                child['code'] = True
                                child['notebook'] = True
                                child['tutorial'] = f'{url_prefix}/{course_file_path}/{title_filename}.md'
                                
                                subprocess.run(["touch", f'./mod{mod}/{title_filename}.ipynb'], check=True)
                                
                            elif v == "code":
                                child['file'] = f'{title_filename}.py'
                                child['title'] = k
                                child['filePath'] = f'{course_file_path}/{title_filename}.py'
                                child['url'] = f'{url_prefix}/{course_file_path}/{title_filename}.py'
                                child['code'] = True
                                child['tutorial'] = f'{url_prefix}/{course_file_path}/{title_filename}.md'
                                
                                subprocess.run(["touch", f'./mod{mod}/{title_filename}.py'], check=True)
                                
                        elif line[0:1] != "_":
                            i_file += 1
                            # new_filename = f'file-{mod}-{i_file}_{line.lower().replace(" ", "_")}'
                            k = line
                            title_filename = f'{k.lower().replace(" ", "_")}'
                            subprocess.run(["touch", f'./mod{mod}/{title_filename}.md'], check=True)
                            
                            child['file'] = f'{title_filename}.md'
                            child['title'] = line
                            child['filePath'] = f'{course_file_path}/{title_filename}.md'
                            child['url'] = f'{url_prefix}/{course_file_path}/{title_filename}.md'
                            child['tutorial'] = f'{url_prefix}/{course_file_path}/{title_filename}.md'
                            
                        # Append other properties
                        elif line[0:1] == '_':
                            other_property = line[1:].split("::")
                            child[other_property[0]] = other_property[1]
            
                        if append_next == 1 and line[0:1] != "_1":
                            append_next = 0
                            sectionObj['children'].append(child)
                                
    return sectionObj, sectionObj['fileName']


# process_playlist_section('./Module-8.bpls')

In [71]:
enrollment_programs = load(f'{project_path}/enrollment-programs.json')

In [72]:
enrollment_programs['programs']['SPR']['videoPlaylist']

[{'fileName': 'Module-0',
  'section': 'Welcome',
  'children': [{'file': 'https://player.vimeo.com/video/779911021?h=11699f372a',
    'title': 'Welcome Video',
    'filePath': 'https://player.vimeo.com/video/779911021?h=11699f372a',
    'url': 'https://player.vimeo.com/video/779911021?h=11699f372a',
    'vimeo': True,
    'duration': ' 02:54',
    'preview': ' True'}]},
 {'fileName': 'Module-1',
  'section': 'Module 1',
  'children': [{'file': 'overview.md',
    'title': 'Overview',
    'filePath': 'bignumber-material/SPR/mod1/overview.md',
    'url': 'http://localhost:3001/bignumber-material/SPR/mod1/overview.md',
    'tutorial': 'http://localhost:3001/bignumber-material/SPR/mod1/overview.md',
    'duration': ' 0mins'},
   {'file': 'https://player.vimeo.com/video/797174163?h=c8f7d92f8d',
    'title': 'Goal of Statistics',
    'filePath': 'https://player.vimeo.com/video/797174163?h=c8f7d92f8d',
    'url': 'https://player.vimeo.com/video/797174163?h=c8f7d92f8d',
    'vimeo': True,
    

### Generate enrollment-programs.json 

In [73]:
# Prepare new playlist dict
new_video_playlist = []

# Directory path
directory = './'

# Process all assessment files (.ba)
# File extension
extension = '.ba'

# Iterate over files in the directory
# for filename in os.listdir(directory):
#     if filename.endswith(extension):
#         file_path = os.path.join(directory, filename)
#         assessmentObj, assessmentFileName = process_assessment(file_path)
#         save(assessmentObj, f'./{assessmentFileName}.json')


# Process all playlist section files (.ba)
# File extension
extension = '.bpls'

# Iterate over files in the directory
files = os.listdir(directory)
files = [filename for filename in files if filename.endswith(extension)]
files = sorted(files)

for filename in files:
    if filename.endswith(extension):
        file_path = os.path.join(directory, filename)
        sectionObj, sectionFileName = process_playlist_section(file_path)
       
        # Populate playlist dict
        new_video_playlist.append(sectionObj)
        
        save(sectionObj, f'./{sectionFileName}.json')

The folder exists.
Saved file
The folder exists.
Saved file
The folder exists.
Saved file
The folder exists.
Saved file
The folder exists.
Saved file
The folder exists.
Saved file
The folder exists.
Saved file
The folder exists.
Saved file
The folder exists.
Saved file
The folder exists.
Saved file


In [74]:
i = 0
for section in new_video_playlist:
    mod_number = section['fileName'][-1]
    mod_folder = f'mod{mod_number}'
    
    j = 0
    for child in section['children']:
        if 'assessment' in child.keys():
            assessmentObj, assessmentFileName = process_assessment(f"./{mod_folder}/{child['file']}")
            
            new_video_playlist[i]['children'][j]['assessment_ref'] = new_video_playlist[i]['children'][j]['assessment']
            new_video_playlist[i]['children'][j]['assessment'] = assessmentObj
            
            save(assessmentObj, f'./{mod_folder}/{child["file"][0:-3]}.json')
        j += 1
    i += 1

Saved file
Saved file
Saved file
Saved file
Saved file
Saved file
Saved file
Saved file
Saved file
Saved file
Saved file
Saved file
Saved file
Saved file
Saved file
Saved file
Saved file
Saved file
Saved file
Saved file
Saved file
Saved file
Saved file
Saved file
Saved file
Saved file
Saved file
Saved file
Saved file
Saved file
Saved file
Saved file
Saved file
Saved file
Saved file
Saved file
Saved file
Saved file
Saved file
Saved file
Saved file
Saved file
Saved file


### Update enrollment-programs.json

In [75]:
enrollment_programs['programs']['SPR']['videoPlaylist'] = new_video_playlist
save(enrollment_programs, f'../enrollment-programs.json')
save(enrollment_programs, f'{project_path}/enrollment-programs.json')

Saved file
Saved file


In [76]:
# json.dumps("true")

In [77]:
!pwd

/Users/danyentezari/Documents/projects_moved/bignumber-material-generator/SPR


In [78]:
!cp -r ./mod* /Users/danyentezari/Documents/projects_moved/bignumber.xyz/public/bignumber-material/SPR

In [79]:
# !rm ./mod*/A*.json

### JSON to BPLS

In [80]:
# project_path = '/Users/danyentezari/Documents/projects_moved/bignumber.xyz/public/bignumber-material/'
# enrollment_programs = load(f'{project_path}/enrollment-programs.json')
# enrollment_programs['programs']['SPR']['videoPlaylist']

In [81]:
# project_path = '/Users/danyentezari/Documents/projects_moved/bignumber.xyz/public/bignumber-material/'
# project_path = './'
# enrollment_programs = load(f'{project_path}/enrollment-programs.json')

# for sectionObj in enrollment_programs['programs']['SPR']['videoPlaylist']:

#     with open(f"./{sectionObj['fileName']}.bpls", 'w') as file:
        
        
#         line_content = f"\
# --- template_start --- \n\
# fileName:: {sectionObj['fileName']} \n\
# section:: {sectionObj['section']} \n\
# --- template_end ---"
        
#         file.write(line_content + '\n\n')
        
#         for childObj in sectionObj['children']:
        
#             # video
#             if childObj['url'][0:24] == "https://player.vimeo.com":
#                 print(f"{childObj['title']}::vimeo('{childObj['url']}')")
#                 line_content = f"{childObj['title']}::vimeo('{childObj['url']}')"                    
#                 file.write(line_content + '\n')
                
#                 if 'description' in childObj.keys():
#                     line_content = f"_description:: {childObj['description']}"
#                     file.write(line_content + '\n')
                    
#                 if 'duration' in childObj.keys():
#                     line_content = f"_duration:: {childObj['duration']}"
#                     file.write(line_content + '\n')
                    
#                 if 'preview' in childObj.keys():
#                     line_content = f"_preview:: {childObj['preview']}"
#                     file.write(line_content + '\n')
                
#                 file.write('\n\n')
                
#             # assessment    
#             if 'assessment' in childObj.keys():   
#                 print(f"{childObj['title']}::ba()")
#                 line_content = f"{childObj['title']}::ba()"
#                 file.write(line_content + '\n')
                
#                 if 'duration' in childObj.keys():
#                     line_content = f"_duration:: {childObj['duration']}"
#                     file.write(line_content + '\n')
                
#                 file.write('\n\n')
            
#             # math_viz    
#             elif 'math_viz' in childObj.keys():   
#                 if childObj['math_viz'] == True:
#                     print(f"{childObj['title']}::math_viz('{childObj['file']}')")
#                     line_content = f"{childObj['title']}::math_viz('{childObj['file']}')"
#                     file.write(line_content + '\n')
                    
#                     if 'duration' in childObj.keys():
#                         line_content = f"_duration:: {childObj['duration']}"
#                         file.write(line_content + '\n')
                    
#                     file.write('\n\n')
                    
                    
#             # notebook    
#             elif 'notebook' in childObj.keys():   
#                 if childObj['notebook'] == True:
#                     print(f"{childObj['title']}::notebook")
#                     line_content = f"{childObj['title']}::notebook"
#                     file.write(line_content + '\n')
                    
#                     if 'duration' in childObj.keys():
#                         line_content = f"_duration:: {childObj['duration']}"
#                         file.write(line_content + '\n')
                    
#                     file.write('\n\n')
                    
#             # code
#             elif 'code' in childObj.keys():   
#                 if childObj['code'] == True:
#                     print(f"{childObj['title']}::code")
#                     line_content = f"{childObj['title']}::code"
#                     file.write(line_content + '\n')
                    
#                     if 'duration' in childObj.keys():
#                         line_content = f"_duration:: {childObj['duration']}"
#                         file.write(line_content + '\n')
                    
#                     file.write('\n\n')
                    
#             else:
#                 print(f"{childObj['title']}")
#                 line_content = f"{childObj['title']}"
#                 file.write(line_content + '\n')
                
#                 if 'duration' in childObj.keys():
#                         line_content = f"_duration:: {childObj['duration']}"
#                         file.write(line_content + '\n')
                    
#                 file.write('\n\n')
                
                    